In [ ]:
import os
import glob
#import WAPORWA modules
os.chdir(r'D:\Github\WAPORWA\modules') #change to modules path
import WaPOR
import WA
from WA.pickle_basin import pickle_in,pickle_out  

#Read pickle
pickle=glob.glob(r'E:\WA_Sheet1\Main\Niger\*.pickle')[-1]
BASIN=pickle_in(pickle)  

# BASIN Info

In [ ]:
BASIN={'Name':'Niger',   
        'Dir':   r"E:\WA_Sheet1\Niger\Main",
        'time_range':['2009-01-01','2018-12-31'],
        'end_month':'DEC',        
        'geo_data':{
                   'basin':r"E:\WA_Sheet1\Input\Global\Basins\Niger.shp",
#                    'unreserv':r"D:\FAO\WA_Sheet1\Input\Awash\Reservoir\Caddabassa_Lake.shp",
#                    'makereserv':r"D:\FAO\WA_Sheet1\Input\Awash\Reservoir\Tendaho_Reservoir.shp",
                      },
        'global_data':{
                     'grace':r"E:\WA_Sheet1\Input\Global\GRACE\GSFC.glb.200301_201607_v02.4-ICE6G",
                     'grand':r"E:\WA_Sheet1\Input\Global\GRanD\GRanD_reservoirs_v1_1.shp",
                     'wdpa':r"E:\WA_Sheet1\Input\Global\WDPA\WDPA_17countries.shp",
                     },
        'input_data':{
                      'yearly':{
                              'p':[r""+os.path.join(Input_folder,'L1_PCP_A'),
                                   'mm/year','Precipitation'],
                              'et':[r""+os.path.join(Input_folder,'L2_AETI_A'),
                                    'mm/year','Actual Evapotranspiration'],
                              'lcc':[r""+os.path.join(Input_folder,'L2_LCC_A'),
                                     '-','Landcover Class'],
                              'i': [r""+os.path.join(Input_folder,'L2_I_A'),
                                     'mm/year','Interception'],
                                'lu':[r"",
                                     '-','WA+ Landuse Categories'],
                              },
                      'monthly':{
                              'p':[r""+os.path.join(Input_folder,'L1_PCP_M'),
                                   'mm/month','Precipitation'],
                              'et':[r""+os.path.join(Input_folder,'L2_AETI_M'),
                                    'mm/month','Actual Evapotranspiration'],
                              'ret':[r""+os.path.join(Input_folder,'L1_RET_M'),
                                    'mm/month','Reference Evapotranspiration'],
                              'i':[r""+os.path.join(Input_folder,'L2_I_M'),
                                    'mm/month','Interception'],
                              'nRD':[r""+os.path.join(Input_folder,'Rainy_Days'),
                                    'days/month','Number of Rainy Days'],
                              },  
                    'stat':{
                             'thetasat': r"D:\FAO\WA_Sheet1\Global\HiHydroSoils\thetasat_topsoil.tif",   
                            }                                                   
                              },
        'input_ts':{
                'dS':None,
                
                    },
        'main_data':{
                'yearly':{},
                     'monthly':{},
                     'stat':{},
                     }                
                }

if not os.path.exists(Basin['Dir']):
    os.makedirs(Basin['Dir'])
pickle_out(Basin)


# Create netcdf

In [ ]:
from WA.create_NC import main as make_netcdf
from WA.create_NC import _get_lats_lons 
import gdal
import tempfile

#Get inputs for create_NC
cutline=BASIN['geo_data']['basin']
Dir_out=os.path.join(BASIN['Dir'],'data','nc')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)    
template=glob.glob(os.path.join(BASIN['input_data']['yearly']['lcc'][0],'*.tif'))[0]          
    

In [ ]:
### yearly maps
for key in BASIN['input_data']['yearly']:             
    if BASIN['input_data']['yearly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_yearly.nc')        
        dataset={key:[BASIN['input_data']['yearly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['yearly'][key][1],                                 
                        'quantity':BASIN['input_data']['yearly'][key][2],
                        'source': 'WaPOR', 'period':'year'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='year')
        if succes:
            BASIN['main_data']['yearly'][key]=nc_fn
            print('Finished {0}_yearly.nc'.format(key))
            
### monthly maps  
for key in BASIN['input_data']['monthly']:           
    if BASIN['input_data']['monthly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_monthly.nc')                
        dataset={key:[BASIN['input_data']['monthly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['monthly'][key][1],                                 
                        'quantity':BASIN['input_data']['monthly'][key][2],
                        'source': 'WaPOR', 'period':'month'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='month')
        if succes:
            BASIN['main_data']['monthly'][key]=nc_fn
            print('Finished {0}_monthly.nc'.format(key))
  
### static maps
lats, lons, optionsProj, optionsClip =_get_lats_lons(template,cutline)
Dir_out=os.path.join(BASIN['Dir'],'data','stat')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
for key in BASIN['input_data']['stat']:            
    if BASIN['input_data']['stat'][key] is not None:  
        basename= os.path.basename(BASIN['input_data']['stat'][key])
        outfn=os.path.join(Dir_out,basename)  
        temp_file = tempfile.NamedTemporaryFile(suffix='.tif').name               
        gdal.Warp(temp_file, BASIN['input_data']['stat'][key], options = optionsProj)
        gdal.Warp(outfn, temp_file, options = optionsClip)
        os.remove(temp_file)
        BASIN['main_data']['stat'][key]=outfn 
        print('Finished {0}'.format(basename))

pickle_out(BASIN)   